<p align="center">
  <a href="https://imc.uc.cl" target="_blank">
    <img src="attachment:image.png" alt="IMC" width="220">
  </a>
</p>

# IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Profesor:** Rodrigo A. Carrasco <br>
---


## Tarea 04 – Aprendizaje automático

- **Fecha de Entrega:** 13 de noviembre de 2025, a las 23:59.
- **Fecha de publicación**: 30 de octubre de 2025.
- **Formato de entrega:** Notebook ejecutado y comentado (`.ipynb`) en el módulo de Tarea 04 habilitado en Canvas.

## Instrucciones

- Esta Tarea debe desarrollarse de manera totalmente *individual*, de acuerdo a lo establecido en la sección de Integridad Académica en el programa del curso.
- La Tarea debe ser desarrollada en lenguaje de programación Python y la entrega en formato Jupyter Notebook.
- El desarrollo del Notebook debe ser claro y ordenado, incluyendo anotaciones (markdown) y comentarios que permitan seguir fácilmente el código y los pasos implementados a los correctores, y siguiendo buenas prácticas de programación. La presentación y claridad del notebook y código forman parte de la evaluación de la tarea.
- Notebook **autocontenible** que:
   - Ejecute sin errores todas las celdas.
  - Contenga tanto el código como los comentarios y explicaciones necesarias.
  - Incluya visualizaciones claras y correctamente etiquetadas.
- No se aceptarán notebooks con celdas rotas o que dependan de rutas externas no indicadas en la tara.

- Deben hacer sus consultas y comentarios sobre la Tarea a traves del canal de Tareas en eo del curso en Canvas.os.
cteriza.

## 1. Objetivos

- Aplicar los conceptos iniciales de manejo de datos y análisis exploratorio vistos en clases.
- Practicar la lectura, limpieza y manipulación de datos en Python.
- Desarrollar habilidades para visualizar y describir patrones y tendencias en conjuntos de datos reales.
- Fomentar la capacidad de comunicar resultados de forma clara y fundamentada.

### 1.1 Objetivo educacional

Esta Tarea tiene como objetivo que los estudiantes desarrollen la capacidad de manejar algunas de las librerías centrales para el desarrollo de Ciencia de Datos, con foco en la lectura y exploración de datos. 

### 1.2 Pregunta de ciencia de datos

Esta tarea tendrá como objetivo crear un predictor que determine el éxito o fracaso de una expedición a distintas montañas del Himalayas según datos del montañista y de la expedición. Vamos a responder a la pregunta de qué tan bien pueden atributos como año, estación, montaña, edad del montañista, entre otros, predecir el éxito o fracaso de una expedición.

### 1.3 Recomendaciones
- Utiliza las librerías sugeridas en el notebook o justifica brevemente si incorporas otras.
- Revisa que todas las celdas se ejecuten en orden, desde el inicio, sin errores.
- Comenta tu código para explicar qué hace cada sección relevante.
- Asegúrate de que las visualizaciones sean fáciles de interpretar y tengan títulos y etiquetas adecuados.

## 2. Datos

[The Himalayan Database](https://www.himalayandatabase.com/) consiste en una colección de registros de expediciones en la cordillera del Himalaya. Esta contiene información detallada de las cimas más altas del mundo, junto con miles de expediciones realizadas desde 1905 hasta 2024 e información de cada uno de los/las montañistas que han participado en ellas. (El dataset que utilizaremos fue extraído desde: https://mavenanalytics.io/data-playground/himalayan-expeditions)

Descargue el archivo `Himalayan+Expeditions.zip` disponible en el repositorio del curso. Como puede ver, esta información está separada en varios archivos `.csv`. Deberá utilizar las herramientas aprendidas durante el curso para juntar esta información, procesarla y crear un predictor que determine el éxito de una subida.

**Recomendación**: Puede revisar el archivo `himalayan_data_dictionary.csv` para saber el significado de cada una de las columnas de la base de datos.

### 2.1 Librerías

Para esta tarea puede utilizar cualquiera de las librerías previamente utilizadas en el curso. Puede agregar otras si lo estima conveniente para el desarrollo de la Tarea.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# ...agreuge otras librerías si lo considera relevante

## 3. Carga y limpieza (1 punto)

### 3.1 DataFrame unificado (0.2 pts)

Inspeccione los atributos de cada dataset y extraiga la siguiente información de ellos:

1. Información de las montañas:
    * ID de la cima (`peakid`)
    * Nombre común de la montaña (`pkname`)
    * Altura (m) (`heightm`)
    * Cantidad de expediciones exitosas (`success_exp`)
    * Cantidad de expediciones fallidas (`failed_exp`)

2. Información de las expediciones:
    * ID de la expedición (`expid`)
    * Año (`year`)
    * Estación del año (`season`)

3. Información de los montañistas:
    * ID del miembro (`memid`)
    * Sexo (`sex`)
    * Año de nacimiento (`yob`)
    * Nacionalidad (`citizen`)
    * Completó la expedición exitosamente (`msuccess`)

Genere un único DataFrame con todos estos atributos, donde cada fila represente el ascenso de un/a montañista en una expedición. (Ojo que esto no necesariamente significa que cada fila será una persona diferente: si una persona escaló dos cumbres distintas en dos expediciones, esta debería aparecer en su DataFrame como dos filas separadas.)


In [3]:
peaks_df = pd.read_csv('data/peaks.csv')
exped_df = pd.read_csv('data/exped.csv', low_memory=False)
members_df = pd.read_csv('data/members.csv', low_memory=False)

exped_df['is_success'] = exped_df['termreason'].str.startswith('Success', na=False)
peak_stats = exped_df.groupby('peakid')['is_success'].agg(
    success_exp=lambda x: x.sum(),
    failed_exp=lambda x: (x == False).sum()
).reset_index()

peaks_info = peaks_df[['peakid', 'pkname', 'heightm']]

peaks_full_info = pd.merge(peaks_info, peak_stats, on='peakid', how='left')
peaks_full_info[['success_exp', 'failed_exp']] = peaks_full_info[['success_exp', 'failed_exp']].fillna(0).astype(int)

exped_info = exped_df[['expid', 'peakid', 'year', 'season']]

members_info = members_df[['expid', 'membid', 'sex', 'yob', 'citizen', 'msuccess']]
members_info = members_info.rename(columns={'membid': 'memid'})

df_merged = pd.merge(members_info, exped_info, on='expid')
df_final = pd.merge(df_merged, peaks_full_info, on='peakid')

final_cols = [
    'memid', 'sex', 'yob', 'citizen', 'msuccess',
    'expid', 'year', 'season',
    'peakid', 'pkname', 'heightm', 'success_exp', 'failed_exp'
]
df_final = df_final[final_cols]

df_final


,memid,sex,yob,citizen,msuccess,expid,year,season,peakid,pkname,heightm,success_exp,failed_exp
0,2,M,1972.0,Australia,False,AMAD01101,2001,Spring,AMAD,Ama Dablam,6814,1123,453
1,1,M,1968.0,Australia,False,AMAD01101,2001,Spring,AMAD,Ama Dablam,6814,1123,453
2,3,M,1960.0,Australia,False,AMAD01101,2001,Spring,AMAD,Ama Dablam,6814,1123,453
3,4,M,1966.0,Australia,False,AMAD01101,2001,Spring,AMAD,Ama Dablam,6814,1123,453
4,5,F,1970.0,Australia,False,AMAD01101,2001,Spring,AMAD,Ama Dablam,6814,1123,453
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89084,2,M,1944.0,UK,False,SURM74101,1974,Spring,SURM,Surma-Sarovar North,6564,1,3
89085,3,M,NaN,UK,False,SURM74101,1974,Spring,SURM,Surma-Sarovar North,6564,1,3
89086,4,M,1948.0,UK,False,SURM74101,1974,Spring,SURM,Surma-Sarovar North,6564,1,3
89087,5,M,NaN,UK,False,SURM74101,1974,Spring,SURM,Surma-Sarovar North,6564,1,3


### 3.2 Limpieza y transformación (0.2 pts)

Inspeccione y procese su DataFrame para generar uno nuevo llamado `clean_df`. En esta sección usted debe decidir si es necesario:

* Limpiar valores nulos, duplicados o inválidos.
* Imputar valores.
* Transformar tipos de columnas.

Justifique su procesamiento.

In [4]:
# respuestas
df_clean = df_final.copy()
df_clean.isnull().sum()


memid             0
sex               0
yob            5424
citizen           7
msuccess          0
expid             0
year              0
season            0
peakid            0
pkname            0
heightm           0
success_exp       0
failed_exp        0
dtype: int64

Se ve un gran numero de valores nulos sobretodo en el "YOB" los cuales se decidiran quitar ya que son un numero considerablemente alto y no se podra imputar un valor en su lugar.

In [5]:
df_clean.dropna(inplace=True)
df_clean.isnull().sum()

memid          0
sex            0
yob            0
citizen        0
msuccess       0
expid          0
year           0
season         0
peakid         0
pkname         0
heightm        0
success_exp    0
failed_exp     0
dtype: int64

In [6]:
df_clean.duplicated().sum()

np.int64(0)

No se ven filas duplicadas por lo que no se realiza ninguna accion respecto a estas

In [7]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 83665 entries, 0 to 89088
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   memid        83665 non-null  int64  
 1   sex          83665 non-null  object 
 2   yob          83665 non-null  float64
 3   citizen      83665 non-null  object 
 4   msuccess     83665 non-null  bool   
 5   expid        83665 non-null  object 
 6   year         83665 non-null  int64  
 7   season       83665 non-null  object 
 8   peakid       83665 non-null  object 
 9   pkname       83665 non-null  object 
 10  heightm      83665 non-null  int64  
 11  success_exp  83665 non-null  int64  
 12  failed_exp   83665 non-null  int64  
dtypes: bool(1), float64(1), int64(5), object(6)
memory usage: 8.4+ MB


### 3.2 Creación de nuevas columnas (0.6 pts)

Vamos a crear un par de columnas nuevas que nos ayudarán a hacer una análisis exploratorio del dataset.
Utilizando las columnas actuales, genere y agregue las siguientes columnas a su DataFrame:

* `age`: Edad del montañista a la fecha de la expedición.
* `peak_success_p`: Porcentaje de expediciones exitosas de la cima respectiva.

De ser necesario, aplique procesos de limpieza y transformación a estas columnas también (asegúrese de que todos los valores sean coherentes).

In [ ]:
df_picos = pd.read_csv('data/peaks.csv', on_bad_lines='skip')
df_clean = df_picos.copy()

# Columnas a eliminar (esta vez, nos aseguramos de NO eliminar 'pexpid')
columnas_a_eliminar = [
    'pkname2', 'phost', 'pcountry', 'psummiters', 'psmtnote'
]
df_clean.drop(columns=columnas_a_eliminar, inplace=True)
df_clean.drop_duplicates(inplace=True)

# Conversión de tipos
columnas_numericas = ['pyear', 'pmonth', 'pday']
for col in columnas_numericas:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

columnas_booleanas = ['open', 'unlisted', 'trekking']
for col in columnas_booleanas:
    df_clean[col] = df_clean[col].astype('bool')

print("'df_clean' recreado. Ahora 'pexpid' está presente.")


# --- 2. Añadir la tasa de éxito ('peak_success_p') ---
print("\nPaso 2: Añadiendo 'peak_success_p'...")
df_exp_stats = pd.read_csv('data/peaks_with_exp.csv')
df_exp_stats['peak_success_p'] = (df_exp_stats['success_exp'] / (df_exp_stats['success_exp'] + df_exp_stats['failed_exp'])) * 100
df_exp_stats['peak_success_p'].fillna(0, inplace=True)
df_clean = pd.merge(df_clean, df_exp_stats[['peakid', 'peak_success_p']], on='peakid', how='left')
print("'peak_success_p' añadido a 'df_clean'.")


# --- 3. Crear 'df_con_edad' con la edad individual ---
print("\nPaso 3: Creando 'df_con_edad' con la edad de cada miembro...")
df_members = pd.read_csv('data/members.csv', on_bad_lines='skip')
df_exped = pd.read_csv('data/exped.csv')

# Calcular edad de cada miembro
df_members_rel = df_members[['expid', 'yob']]
df_exped_rel = df_exped[['expid', 'year']]
df_member_exped = pd.merge(df_members_rel, df_exped_rel, on='expid')
df_member_exped['yob'] = pd.to_numeric(df_member_exped['yob'], errors='coerce')
df_member_exped['year'] = pd.to_numeric(df_member_exped['year'], errors='coerce')
df_member_exped['age'] = df_member_exped['year'] - df_member_exped['yob']

# Preparar DataFrame de edades
df_member_age = df_member_exped[['expid', 'age']].dropna()
df_member_age['age'] = df_member_age['age'].astype(int)

# Ahora el merge funcionará porque 'pexpid' existe en 'df_clean'
df_con_edad = pd.merge(
    df_clean,
    df_member_age,
    left_on='pexpid',
    right_on='expid',
    how='left'
)


'df_clean' recreado. Ahora 'pexpid' está presente.

Paso 2: Añadiendo 'peak_success_p'...
'peak_success_p' añadido a 'df_clean'.

Paso 3: Creando 'df_con_edad' con la edad de cada miembro...


C:\Users\gasxd\AppData\Local\Temp\ipykernel_20704\1116119176.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_exp_stats['peak_success_p'].fillna(0, inplace=True)
C:\Users\gasxd\AppData\Local\Temp\ipykernel_20704\1116119176.py:34: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  df_members = pd.read_csv('data/members.csv', on_bad_lines='skip')
C:\Users\gasxd\AppData\Local\Temp\ipykernel_20704\1116119176.py:35: DtypeWarning: Columns (8,18,19) have mixed types. Specify 

In [10]:
df_clean.head()

,peakid,pkname,location,heightm,heightf,himal,region,open,unlisted,trekking,trekyear,restrict,pstatus,pyear,pseason,pmonth,pday,pexpid,peak_success_p
0,ACHN,Aichyn,Chandi Himal (SW of Changwathang),6055,19865,Nalakankar/Chandi/Changla,Kanjiroba-Far West,True,False,False,NaN,Opened in 2014,Climbed,2015.0,Autumn,NaN,3.0,ACHN15301,66.666667
1,AMAD,Ama Dablam,Khumbu Himal,6814,22356,Khumbu,Khumbu-Rolwaling-Makalu,True,False,False,NaN,NaN,Climbed,1961.0,Spring,NaN,13.0,AMAD61101,71.256345
2,AMOT,Amotsang,Damodar Himal (NW of Pokharhan),6393,20974,Damodar,Annapurna-Damodar-Peri,True,False,False,NaN,Opened in 2002,Climbed,2019.0,Autumn,NaN,24.0,AMOT19301,33.333333
3,AMPG,Amphu Gyabjen,Khumbu Himal (N of Ama Dablam),5630,18471,Khumbu,Khumbu-Rolwaling-Makalu,True,False,False,NaN,Opened in 2002,Climbed,1953.0,Spring,NaN,11.0,AMPG53101,75.000000
4,AMPH,Amphu I,"Khumbu Himal (E of Amphu Laptsa, W of Baruntse)",6740,22113,Khumbu,Khumbu-Rolwaling-Makalu,True,False,False,NaN,Opened in 2002,Climbed,2013.0,Autumn,NaN,9.0,AMPH13301,50.000000


In [ ]:
df_members = pd.read_csv('data/members.csv', on_bad_lines='skip')
df_exped = pd.read_csv('data/exped.csv')

# 1. Cruzar members.csv con exped.csv para obtener el peakid y el año de la expedición.
df_members_rel = df_members[['expid', 'yob']]
df_exped_rel = df_exped[['expid', 'peakid', 'year']]
df_member_exped = pd.merge(df_members_rel, df_exped_rel, on='expid')

# 2. Calcular la edad de cada miembro.
# Nos aseguramos de que las columnas sean numéricas.
df_member_exped['yob'] = pd.to_numeric(df_member_exped['yob'], errors='coerce')
df_member_exped['year'] = pd.to_numeric(df_member_exped['year'], errors='coerce')
df_member_exped['age'] = df_member_exped['year'] - df_member_exped['yob']

# 3. Calcular la edad promedio para cada pico.
df_peak_avg_age = df_member_exped.groupby('peakid')['age'].mean().reset_index()

# 4. Fusionar la edad promedio con el DataFrame df_clean.
# El error MergeError ocurre al re-ejecutar la celda, porque la columna 'age' ya existe.
# Para solucionarlo, renombramos la columna de edad promedio y la eliminamos de df_clean si ya existe.
df_peak_avg_age.rename(columns={'age': 'edad_promedio_pico'}, inplace=True)

if 'edad_promedio_pico' in df_clean.columns:
    df_clean.drop('edad_promedio_pico', axis=1, inplace=True)

df_clean = pd.merge(df_clean, df_peak_avg_age, on='peakid', how='left')


C:\Users\gasxd\AppData\Local\Temp\ipykernel_15748\235853403.py:1: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  df_members = pd.read_csv('data/members.csv', on_bad_lines='skip')
C:\Users\gasxd\AppData\Local\Temp\ipykernel_15748\235853403.py:2: DtypeWarning: Columns (8,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_exped = pd.read_csv('data/exped.csv')


In [ ]:
df_clean.head()

,memid,sex,yob,citizen,msuccess,expid_x,year,season,peakid,pkname,heightm,success_exp,failed_exp,peak_success_p,expid_y,age_x,age_y,age,edad_promedio_pico
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACHN15301,22.600000,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACHN15302,49.181818,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACHN18301,44.555556,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AMAD00101,31.200000,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AMAD00102,39.000000,NaN,NaN,NaN


## 4. EDA (2.5 puntos)

Con el dataset obtenido, responda las siguientes preguntas. Para cada una de ellas, realice al menos un gráfico que permita visualizar el comportamiento de su dataset.

a. **(0.5 pts)** ¿Cuántos ascensos exitosos y fallidos hay en su dataset? ¿A qué porcentaje del total corresponde cada categoría?

b. **(0.5 pts)** ¿Cómo se distribuyen las edades de los/las montañistas en la base de datos?

c. **(0.5 pts)** ¿Cómo se distribuye la cantidad de ascensos según estación del año? ¿Cuál es el porcentaje de éxito por estación? Muestre la cantidad de filas de ascensos exitosas y fallidas para cada estación.

d. **(0.5 pts)** ¿Cómo cambia la cantidad de ascensos en el tiempo? Muestre la cantidad de ascensos exitosos y fallidos por año.

e. **(0.5 pts)** ¿Existe una correlación entre el porcentaje de éxito de ascenso y la altura de una cumbre? Haga un scatterplot para todas las cumbres que hayan sido escaladas por al menos 10 expediciones en total (tanto fallidas como exitosas).

In [ ]:
# respuestas

Respuestas:

## 5. Predictor (2.5 puntos)

Ahora veremos si la información que tenemos nos permite predecir el éxito de un ascenso por alguna persona en particular. Primero, defina su **X** (matriz de atributos) e **y** (etiqueta a predecir) de la siguiente manera:

* **X**: `age`, `sex`, `citizen`, `season`, `heightm`, `year`, `success_exp`, `failed_exp`
* **y**: `msuccess`

Luego, realice los pasos a continuación:

**5.1 Codificación y normalización (1 pto)**
* Codifique todas las columnas categóricas, de manera que su matriz **X** solo tenga valores numéricos.
* Divida sus datos en sets de entrenamiento y testeo. Use una división de 70-30.
* Normalice sus matrices de atributos. Justifique su método de normalización.

**5.2 Creación y entrenamiento de modelos (1 pto)**
* Cree y entrene los siguientes modelos:
    * Un clasificador KNN
    * Un árbol de decisión
    * Un clasificador de regresión logística
    * Un clasificador random forest **(Opcional)**

    Explique qué hiperparámetros utilizó para la creación de cada uno de ellos y qué valores le asignó.

* Finalmente, evalúe todos sus modelos con el set de testeo. Para cada uno, muestre
    * Accuracy
    * Precisión
    * Recall
    * Matriz de confusión

**5.3 Análisis de resultados (0.5 pts)**

Interprete, analice y comente sus resultados. 
* ¿Qué tendencias puede observar? ¿Se diferencian de alguna forma las predicciones entre un modelo y otro? 
* ¿Fue posible construir un predictor efectivo? Justique.



In [ ]:
# respuestas

Respuestas: